# Text to SQL Similarity

The purpose of this task is to asses your independent work and familiarity with data analysis, Machine Learning and Natural Language Processing techniques. Feel free to use any libraries you want, any models necessary and to define any additional functions you might need. 

You are given a dataset with questions and their corresponding SQL queries.

Tasks:
1. Define 2 different similarity metrics between a question and a SQL query (your choice - be creative). The metrics should use different techniques and be based on different models.
2. Analyze the similarity metrics you defined above.
    
    * Define plots to visualize the performance of the similarity metrics (your choice - be creative)
    * Define a threshold (and how to choose it) for the similarity metrics to determine if a question is similar to a SQL query.
    * Calculate the precision, recall, and F1 score per similarity metric (for the threshold you defined) or other metrics of your choice that will help you compare between the metrics.
3. Summarize your findings and conclusions.




In [67]:
import pandas as pd
import numpy as np
import os

from tabulate import tabulate
from dotenv import load_dotenv
from datasets import Dataset
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas import evaluate

load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [68]:
df = pd.read_csv('data.csv')
df
 

,inquiry_id,question,sql
0,inquiry_MTzoIbMP,In what countries are most of our jobs offered?,"SELECT t.country_name AS Country, COUNT(*) AS ..."
1,inquiry_aiTiTTpt,What is the churn rate of a scout after trying...,"WITH UserData AS (\n SELECT\n ur.user_id,\..."
2,inquiry_nzDQlAKx,What is the average time to fulfill a job?,"WITH hiredJobIds AS( SELECT job_id, date FROM(..."
3,inquiry_gJUTBJjF,How does the average time to fill a job vary b...,"WITH hiredJobs AS( SELECT *, FROM( SELECT even..."
4,inquiry_wTCHJIzI,What is the trend in activation time for scouts?,\nDECLARE start_date DATE DEFAULT '2024-11-22'...
...,...,...,...
70,inquiry_dXUxZMKU,Which jobs have been posted to date?,"SELECT j.id AS job_id, c.name AS company_name,..."
71,inquiry_wMMvZVYt,What is the average number of candidates rejec...,"SELECT u.name, COUNT(DISTINCT ja.id) AS jobs_r..."
72,inquiry_iIqSbznQ,How Often Is A Job Posted?,WITH JobPostDates AS( SELECT DATE(publi...
73,inquiry_zZXzZGVE,How Many Candidates Have Been Placed In Jobs S...,SELECT count(distinct id) AS `Placed_Candidate...


In [69]:

print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")
print(df["inquiry_id"].unique().size)
print(df["question"].unique().size)
print(df["sql"].unique().size)

df= df.iloc[:2]




Number of rows: 75
Number of columns: 3
75
75
74


In [70]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else ("" if pd.isnull(x) else x))
print (df.isnull().sum())

inquiry_id    0
question      0
sql           0
dtype: int64


C:\Users\MSI KATANA\AppData\Local\Temp\ipykernel_17376\760036421.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else ("" if pd.isnull(x) else x))


In [71]:
id_map = {}
counter = 1

# Step 2: Generate IDs for each row based on inquiry_id
def get_or_assign_id(inquiry):
    global counter
    if inquiry not in id_map:
        id_map[inquiry] = counter
        counter += 1
    return id_map[inquiry]

# Add the new 'id' column
df["id"] = df["inquiry_id"].apply(get_or_assign_id)

column_order = ["id", "inquiry_id", "question", "sql"]
df = df[column_order]
print(tabulate(df, headers='keys', tablefmt='pretty', showindex=False))

+----+------------------+------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [74]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

df["retrieved_contexts"] = df["sql"].apply(lambda x: [x])  
df["reference"] = df["question"]
df["response"] = " "
 
dataset = Dataset.from_pandas(df)

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4"))
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())
 
metrics = [
    LLMContextRecall(llm=evaluator_llm),
    FactualCorrectness(llm=evaluator_llm),
    Faithfulness(llm=evaluator_llm),
    SemanticSimilarity(embeddings=evaluator_embeddings),
    context_precision ,
    answer_relevancy,
    context_recall  
]


results = evaluate(dataset=dataset, metrics=metrics)
df_eval = results.to_pandas()
 


Evaluating: 100%|██████████| 14/14 [00:14<00:00,  1.03s/it]


In [75]:
df_eval

,user_input,retrieved_contexts,response,reference,context_recall,factual_correctness,faithfulness,semantic_similarity,context_precision,answer_relevancy
0,In what countries are most of our jobs offered?,"[SELECT t.country_name AS Country, COUNT(*) AS...",,In what countries are most of our jobs offered?,0.0,NaN,NaN,0.739802,0.0,0.0
1,What is the churn rate of a scout after trying...,"[WITH UserData AS (\n SELECT\n ur.user_id,...",,What is the churn rate of a scout after trying...,0.0,NaN,NaN,0.714260,0.0,0.0


# Metric 1:

# Threshold Selection Metric 1:


# Metric 2:

# Threshold Selection Metric 2:

# Metrics Analysis:

# Conclusion and Findings: